# Pryngles module: System

In [1]:
from pryngles import *

## External modules

In [2]:
import rebound as rb

## System Class

This is the most important class in the whole package.  This class allows to create the planetary system and manipulate it.

In [3]:
"""
#Create a simple system
#Once you create a system, a null spangler is created 

sys.set_observer(n_obs=[1,1,0],alpha_obs=0)

#Add star (by default, m = 1)
S=sys.add()

#Add planet, when an object is added, it is automatically spangled
P=sys.add("Planet",radius=0.1,m=1e-3,a=1,e=0.2)

#Add moon: orbital elements are respect to equatorial plane of the primary
M=sys.add("Planet",primary=P,radius=0.01,m=1e-7,a=0.1,e=0.01)

#Add ring system
R=sys.add("Ring",primary=P,fi=1.5,fe=2.5,albedo_gray_normal=0.5,tau_gray_optical=3)

#If you change the number of spangles of an object the spanglers are reset
sys.update_body(R,nspangles=800)

#Each time an object is updated, the spangler should be rejoined and the simulation reset.

#Spangle 
#sys.spangle_system()

#You may check separately the properties of each object
R.spangler.plot3d()
R.spangler.plot_obs()

#Plot
sys.spangler.plot3d()
sys.spangler.plot_obs()
""";

In [4]:
System_doc=\
f"""
Creates a planetary system.

Initialization attributes:

    units: list of strings, default = ['au','msun','yr']:
        Units used in calculations following the conventions and signs of rebound.
        The order SHOULD always be MKS: length, mass, time (in that order)

Derived attributes:

    sim: Simulation:
        REBOUND Simulation object.
        
    ul, um, ut: float [SI units]
        Value of the conversion factors for each unit.
        
    G: float [ul^3/ut^2/um]
        Value of the gravitational constant.

    hashes: list:
        List of hashes of bodies in the system.
        
    stars, planets, rings: lists:
        List of the corresponding kind of object in the system.
        
    nstars, nplanets, nrings, nobservers, nbodies: int
        Number of each kind of body in the system and of all bodies in the system.

Examples:

    #Create a system
    sys=System(units=["au","msun","yr"])
    sys.sim.integrator='wahfast'
    sys.sim.dt=0.01
    
    #Add star (by default, m = 1)
    S=sys.add()

    #Add planet, when an object is added, it is automatically spangled
    P=sys.add("Planet",radius=0.1,m=1e-3,a=1,e=0.2)

    #Add moon: orbital elements are respect to equatorial plane of the primary
    M=sys.add("Planet",primary=P,radius=0.01,m=1e-7,a=0.1,e=0.01)

    #Add ring system
    R=sys.add("Ring",primary=P,fi=1.5,fe=2.5,albedo_gray_normal=0.5,tau_gray_optical=3)

""";

In [5]:
class System(PrynglesCommon):
    
    def __init__(self,
                 units=['au','msun','yr'],
                ):
        
        #Rebound simulation
        self.sim=rb.Simulation()
        
        #Update rebound units
        self.update_units(units)
        
        #Bodies
        self.bodies=[]
        self.nbodies=0
        
        #Update system
        self._update_system()
        
    def update_units(self,units):
        """Update units of the system
        """
        self.units=units
        
        #Units
        self._ul,self._um,self._ut=self.units
        self.sim.units=self.units
        
        #Canonical units of the system
        self.ul=rb.units.convert_length(1,self._ul,"m")
        self.um=rb.units.convert_mass(1,self._um,"kg")
        self.ut=np.sqrt(self.sim.G*self.ul**3/(self.um*GSI))
        
    def _update_system(self):
        pass

System.__doc__=System_doc

In [6]:
if IN_JUPYTER:
    def test_system_init(self):
        
        Verbose.VERBOSITY=VERB_ALL
        
        sys=System()
        print(sys.nbodies)
        print(sys.sim.G)
        print(sys.ul,sys.um,sys.ut)
        
        sys=System(units=['m','kg','s'])
        print(sys.nbodies)
        print(sys.sim.G)
        print(sys.ul,sys.um,sys.ut)
        
        print(sys)

        Verbose.VERBOSITY=VERB_NONE

    class Test(unittest.TestCase):pass    
    Test.test_system_init=test_system_init
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

0
39.476926421373
149597870700.0 1.9884754159665356e+30 31557600.0
0
6.67408e-11
1.0 1.0 1.0
{'sim': <rebound.simulation.Simulation object at 0x7fa671334240, N=0, t=0.0>, 'units': ['m', 'kg', 's'], 'ul': 1.0, 'um': 1.0, 'ut': 1.0, 'bodies': [], 'nbodies': 0}



----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [61]:
def add(self,kind="Star",primary=None,**props):
    """Add an object to the system
    
    Examples:
    
        sys=System()
        S=sys.add("Star",m=2)
    
    Parameters:
    
        kind: string, default = "Star":
            Kind of object: Star, Planet, Ring (see BODY_KINDS).
    
        primary: Body, default = None:
            Primary object of the body.

        props: dictionary:
            List of properties of the body
            
    Returns:
        
        Body
            Body added to the system.
    """
    if kind is None:
        raise AssertionError("You must provide a valid object kind (Star, Planet, Ring).")

    if kind not in BODY_KINDS:
        raise ValueError(f"Object kind '{kind}' is not recognized.")

    if kind=="Planet":
        print(primary)
        Planet(primary=primary)
    print(f"Object {kind}")
    exec(f"self.bodies+=[{kind}(primary=primary,**props)]")
    self.nbodies=len(self.bodies)
    
    #Las body added
    body=self.bodies[-1]
    
    if kind != "Ring":
        #Add body to simulation
        rb_add_options={k:v for k,v in body.__dict__.items() if k in REBOUND_ORBITAL_PROPERTIES}
        verbose(VERB_VERIFY,f"Adding object with hash {body.hash}")
        if primary:
            rb_add_options.update(primary=primary.hash)
        rb_add_options.update(hash=body.hash)
        self.sim.add(**rb_add_options)
    
    return body
    
System.add=add

In [65]:
"""
sys=System()
S=sys.add(m=8,radius=4)
P=sys.add("Planet",primary=S,a=0.1,e=0.2,radius=2)
R=sys.add("Ring",primary=P,fi=1.3,fe=2.3)
for key in sys.sim.particles.keys():
    print(key)
#""";

[autoreload of pryngles.system failed: Traceback (most recent call last):
  File "/Users/jorgezuluagacallejas/opt/anaconda3/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/jorgezuluagacallejas/opt/anaconda3/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/Users/jorgezuluagacallejas/opt/anaconda3/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/Users/jorgezuluagacallejas/Dropbox/MiInvestigacion/PapersEnProceso/Exorings/pryngles/src/pryngles/system.py", line 217, in <module>
    type(S)
NameError: name 'S' is not defined
]


In [61]:
if IN_JUPYTER:
    def test_system_add(self):
        
        Verbose.VERBOSITY=VERB_ALL
        
        sys=System()
        S=sys.add(m=8,radius=4)
        print(S)
        P=sys.add("Planet",primary=S,a=0.1,e=0.2,radius=2)
        print(P)
        R=sys.add("Ring",primary=P,fi=1.3,fe=2.3)
        print(R)

        Verbose.VERBOSITY=VERB_NONE
                
    class Test(unittest.TestCase):pass    
    Test.test_system_add=test_system_add
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

      VERB3::update_body:: Updating Body
      VERB3::_update_properties:: Updating properties of Body
      VERB3::_update_star_properties:: Updating properties of Star
{'kind': 'Star', 'hash': 8769269061295, 'primary': None, 'm': 8, 'radius': 4, 'prot': 1, 'i': 0, 'roll': 0, 'alpha': 0, 't0': 0, 'nspangles': 1000, 'type_spangle': 3, 'geometry': 'sphere', 'geometry_args': {}, 'seed': 0, 'preset': True, 'limb_coeffs': [], 'wrot': 6.283185307179586, 'n_equ': array([6.123234e-17, 0.000000e+00, 1.000000e+00]), 'norm_limb_darkening': 3.141592653589793}
      VERB3::_update_childs:: Add child 8769269057040 to body Star (8769269061295)
      VERB3::_update_primary:: Add primary 8769269061295 to body Planet (8769269057040)
      VERB3::update_body:: Updating Body
      VERB3::_update_properties:: Updating properties of Body
      VERB3::update_planet:: Updating Planet
      VERB3::update_body:: Updating Body
      VERB3::_update_properties:: Updating properties of Body
      VERB3::_update_pl


----------------------------------------------------------------------
Ran 1 test in 0.163s

OK


In [32]:
def remove(self,body_hash):
    """Remove a body from a system.

    Parameters:
        body_hash: string
            Hash of the body to remove
    
    Notes: 
        Remove eliminate body and all the childs and the childs of the childs.

    Example:
        sys=System()
        S=sys.add(kind="Star",orbit=dict(m=2))
        sys.remove(body_hash=S.hash)
        
    """
    if body_hash in self.hashes:
        obj=self.hashes[body_hash]
        lkind=obj.kind.lower()

        #Remove child objects
        for child in obj.childs:
            if child.hash in self.hashes:
                self.remove(child.hash)

        #Remove object
        exec(f"self.{lkind}s.remove(obj)")

        #Remove hash from list
        self.hashes.pop(body_hash)

        #Update system
        self._update_system()
    else:
        raise ValueError("No object with hash 'body_hash' in the system")
System.remove=remove

In [24]:
if IN_JUPYTER:
    def test_system_remove(self):
        sys=System()
        S=sys.add(kind="Star",orbit=dict(m=2))
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)

        sys.remove(body_hash=S.hash)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        
        sys=System()
        S=sys.add(kind="Star")
        P=sys.add(kind="Planet",primary=S)
        R=sys.add(kind="Ring",primary=P)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)

        sys.remove(body_hash=S.hash)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)

        sys=System()
        S=sys.add(kind="Star")
        P=sys.add(kind="Planet",primary=S)
        R=sys.add(kind="Ring",primary=P)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)

        sys.remove(body_hash=P.hash)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        
        sys=System()
        S=sys.add(kind="Star")
        P=sys.add(kind="Planet",primary=S)
        R=sys.add(kind="Ring",primary=P)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)

        sys.remove(body_hash=R.hash)
        print(sys.hashes)
        print(sys.nbodies,sys.nstars,sys.nplanets,sys.nrings,sys.nobservers)
        
        """
        self.assertEqual(np.isclose([P.physics.wrot],
                                    [2*np.pi/PlanetDefaults.physics["prot"]],
                                    rtol=1e-7),
                         [True]*1)
        #Check exception: primary could not be different from None or Body
        self.assertRaises(AssertionError,lambda:Observer(primary="Nada"))
        """
        
    class Test(unittest.TestCase):pass    
    Test.test_system_remove=test_system_remove
    unittest.main(argv=['first-arg-is-ignored'],exit=False)

.

{'8767353992208': <pryngles.star.Star object at 0x7f94f0534100>}
1 1 0 0 0
{}
0 0 0 0 0
{'8767372253133': <pryngles.star.Star object at 0x7f9501bd7cd0>, '8767372435953': <pryngles.planet.Planet object at 0x7f9501ea1f10>, '8767372253040': <pryngles.ring.Ring object at 0x7f9501bd7700>}
3 1 1 1 0
{}
0 0 0 0 0
{'8767372435884': <pryngles.star.Star object at 0x7f9501ea1ac0>, '8767372253097': <pryngles.planet.Planet object at 0x7f9501bd7a90>, '8767372443612': <pryngles.ring.Ring object at 0x7f9501ebfdc0>}
3 1 1 1 0
{'8767372435884': <pryngles.star.Star object at 0x7f9501ea1ac0>}
1 1 0 0 0
{'8767372443633': <pryngles.star.Star object at 0x7f9501ebff10>, '8767372435863': <pryngles.planet.Planet object at 0x7f9501ea1970>, '8767372267045': <pryngles.ring.Ring object at 0x7f9501c0e250>}
3 1 1 1 0
{'8767372443633': <pryngles.star.Star object at 0x7f9501ebff10>, '8767372435863': <pryngles.planet.Planet object at 0x7f9501ea1970>}
2 1 1 0 0



----------------------------------------------------------------------
Ran 1 test in 0.008s

OK


In [3]:
def spangle_system():
    pass

In [5]:
def integrate_system(t=0):
    pass

--End--

In [ ]:
#Create a simple system
sys=System(units=["au","msun","yr"])
sys.sim.integrator='wahfast'
sys.sim.dt=0.01
#Once you create a system, a null spangler is created 
"""
You may set the observer from the very beginning
"""
sys.set_observer(n_obs=[1,1,0],alpha_obs=0)

#Add star (by default, m = 1)
S=sys.add()

#Add planet, when an object is added, it is automatically spangled
P=sys.add("Planet",radius=0.1,m=1e-3,a=1,e=0.2)

#Add moon: orbital elements are respect to equatorial plane of the primary
M=sys.add("Planet",primary=P,radius=0.01,m=1e-7,a=0.1,e=0.01)

#Add ring system
R=sys.add("Ring",primary=P,fi=1.5,fe=2.5,albedo_gray_normal=0.5,tau_gray_optical=3)

#If you change the number of spangles of an object the spanglers are reset
sys.update_body(R,nspangles=800)

#Each time an object is updated, the spangler should be rejoined and the simulation reset.

#Spangle 
#sys.spangle_system()

#You may check separately the properties of each object
R.spangler.plot3d()
R.spangler.plot_obs()

#Plot
sys.spangler.plot3d()
sys.spangler.plot_obs()

In [ ]:
print(R)

In [ ]:
#Update properties of objects, even after the system is spangled
sys.update_body(S,limb_coeffs=[0.5,0.2])
sys.update_body(R,i=30*Consts.deg,roll=60*Consts.deg)
sys.update_body(P,Prot=24*Const.hour/sys.ul)

In [ ]:
sys.set_observer(n_obs=[1,1,1],alpha_obs=0)

In [ ]:
#When you integrate you update the position of the bodies, recalculate light sources, observer configuration and 
sys.sim_integrate(0.5)

sys.spangler.plot3d()
sys.spangler.plot_obs()

In [ ]:
sys.sim_reset()

In [ ]:
#Update positions: in this way you can move the planet, for instance, around the star
sys.update_body(R,M=30*Consts.deg)

In [ ]:
"""
You may integrate without calculating photometry

Once you're ready calculate photometry
"""
sys.calc_photometry()

In [ ]:
"""
This routine only update the optical states of the spangles
"""
sys._update_states()

"""
This routine update temperatures.  This take into account the illumination history of the spangles
"""
sys._update_temperatures()